In [1]:
defaultFolder="../input/aptos2019-blindness-detection"

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import tensorflow as tf
#from tensorflow import keras
import tensorflow.keras.preprocessing.image as image
import IPython.display as display
import matplotlib.image as mping;
import matplotlib.pyplot as plt;
import scipy.ndimage
import imageio
import seaborn as sns
import cv2
#from tensorflow.keras.applications import resnet50,vgg16,DenseNet121
#from tensorflow.keras import callbacks
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras import layers 
from sklearn.model_selection import train_test_split
from efficientnet import *
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import VGG16,VGG19, DenseNet121,DenseNet169,ResNet50
from keras.callbacks import Callback, ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
from keras import layers
from tqdm import tqdm, tqdm_notebook
from sklearn.metrics import cohen_kappa_score, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras.models import load_model,Input
#from tensorflow.keras.applications.resnet50 import preprocess_input
#from tensorflow.keras.applications.vgg19 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir('../input'))

# Any results you write to the current directory are saved as output.

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

['aptosweightslb78', 'efficientnet-keras-weights-b0b5', 'b5-2-of-mixed-aptos-2019-blindness-detectio-eff', 'b5-cv-of-mixed-aptos-2019-blindness-detection', 'b3-of-mixed-aptos-2019-blindness-detectio-efficent', 'aptos2019-blindness-detection', 'b4-mixed-aptos-2019-blindness-detection-effic', 'mixed-aptos-2019-blindness-inceptionresent', 'mixed-aptos-2019-blindness-detection-efficientnet']


Using TensorFlow backend.


In [3]:
defaultFolder="../input/aptos2019-blindness-detection"

In [4]:
train_data=pd.read_csv(defaultFolder+'/train.csv')
test_data=pd.read_csv(defaultFolder+'/test.csv')

In [5]:
y_train=train_data['diagnosis']
train_data.head()

,id_code,diagnosis
0,000c1434d8d7,2
1,001639a390f0,4
2,0024cdab0c1e,1
3,002c21358ce6,0
4,005b95c28852,0


In [6]:
train_data.shape

(3662, 2)

In [7]:
IMAGE_SIZE=(256, 256)
desired_size=256
train_data.dtypes

id_code      object
diagnosis     int64
dtype: object

In [8]:
train_data["id_code"]=train_data["id_code"].apply(lambda x:x+".png")
train_data['diagnosis'] = train_data['diagnosis'].astype(str)
lbls = list(map(str, range(5)))

In [9]:
test_data["id_code"]=test_data["id_code"].apply(lambda x:x+".png")

In [10]:
def crop_image1(img,tol=7):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img
    
    
def circle_crop(img, sigmaX=10):   
    """
    Create circular crop around image centre    
    """    
    
    #img = cv2.imread(img)
    img = crop_image_from_gray(img)    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    height, width, depth = img.shape
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    img=cv2.addWeighted ( img,4, cv2.GaussianBlur( img , (0,0) , sigmaX) ,-4 ,128)
    return img

In [11]:
def preprocess_Color_imageArr(image_arr,sigmaX=40):
    """
    Create circular crop around image centre    
    """    
    #image_arr = cv2.imread(image_path)
    img = crop_image_from_gray(image_arr)    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    img=cv2.addWeighted ( img,4, cv2.GaussianBlur( img , (0,0) , sigmaX) ,-4 ,120)  
    img = cv2.resize(img, (IMAGE_SIZE))
    return img

In [12]:
print(os.listdir('../input/aptosweightslb78'))

['aptos_efficientnet-b5_model.best.h5', 'aptos_efficientnet-b5_weights.best.h5']


In [13]:
model=load_model('../input/aptosweightslb78/aptos_efficientnet-b5_model.best.h5')
model.load_weights('../input/aptosweightslb78/aptos_efficientnet-b5_weights.best.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Model)              (None, 8, 8, 2048)        28513520  
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 8, 2048)        0         
_________________________________________________________________
dense_1 (Dense)              (None, 8, 8, 64)          131136    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 64)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total para

In [14]:
%%time
test_datagen=ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess_Color_imageArr)
test_generator=test_datagen.flow_from_dataframe(  
        dataframe=test_data,
        directory = "../input/aptos2019-blindness-detection/test_images",    
        x_col="id_code",
        target_size = IMAGE_SIZE,
        batch_size = 1,
        shuffle = False,
        color_mode='rgb',
        class_mode = None
        )

Found 1928 validated image filenames.
CPU times: user 44 ms, sys: 112 ms, total: 156 ms
Wall time: 614 ms


In [15]:
%%time
y_test=model.predict_generator(test_generator, steps = len(test_generator.filenames))

CPU times: user 41min 33s, sys: 1min 49s, total: 43min 22s
Wall time: 12min 39s


In [16]:
coef = [0.5, 1.5, 2.5, 3.5]

for i, pred in enumerate(y_test):
    if pred < coef[0]:
        y_test[i] = 0
    elif pred >= coef[0] and pred < coef[1]:
        y_test[i] = 1
    elif pred >= coef[1] and pred < coef[2]:
        y_test[i] = 2
    elif pred >= coef[2] and pred < coef[3]:
        y_test[i] = 3
    else:
        y_test[i] = 4

In [17]:
y_test

array([[2.],
       [3.],
       [2.],
       ...,
       [2.],
       [2.],
       [2.]], dtype=float32)

In [18]:
test_data['diagnosis'] = y_test.astype(int)
test_data['id_code'] = test_data['id_code'].map(lambda x: str(x)[:-4])
test_data.to_csv("submission.csv",index=False)
test_data.head()

,id_code,diagnosis
0,0005cfc8afb6,2
1,003f0afdcd15,3
2,006efc72b638,2
3,00836aaacf06,2
4,009245722fa4,2
